using @datastax to build astradb database to store car details
ask copilot: I want to build a table using datastax astradb to store the car information from my code being the color, brand and condition of the car appearing on the input image. How can I do this?



In [1]:
# Image Analysis with GPT-4o model

from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from dotenv import load_dotenv
import os
import json
import re
import uuid
from openai import AzureOpenAI
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider


def analyse_car_image(image_path):

    USE_AZURE_OPENAI = True
    load_dotenv(".env")

    AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")
    AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
    subscription_key = os.getenv("AZURE_SEARCH_SUB_KEY")
    endpoint = os.getenv("AZURE_SEARCH_SUB_ENDPOINT")

    client = AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        api_version="2024-02-15-preview"
    )

    computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

    # Open the image file and analyze it
    with open(image_path, "rb") as image_file:
        image_analysis = computervision_client.analyze_image_in_stream(image_file, [VisualFeatureTypes.tags, VisualFeatureTypes.color])

    image_tags = ', '.join([tag.name for tag in image_analysis.tags])
    image_colors = image_analysis.color

    # Extract color information
    dominant_colors = ', '.join(image_colors.dominant_colors)
    accent_color = image_colors.accent_color
    dominant_foreground_color = image_colors.dominant_color_foreground
    dominant_background_color = image_colors.dominant_color_background

    image_message = (
        f"The image contains the following elements: {image_tags}. "
        f"The dominant colors in the image are: {dominant_colors}. "
        f"The dominant foreground color is: {dominant_foreground_color}. "
    )

    # Use GPT-4 to generate new prompt or analysis
    messages = [
        {"role": "system", "content": 'Be friendly and descriptive'},
        {"role": "user", "content": image_message},
        {"role": "user", "content": "Based on these elements and colors, return the brand, color, and condition of the car as a JSON object. The condition should refer to the appearance of the car (e.g., good condition, bad condition)."}
    ]

    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=messages,
        temperature=0.14,
        max_tokens=800,
        top_p=0.17,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
    )

    response_content = response.choices[0].message.content

    # Print the full response (for debugging)
    print("Full response:", response_content)

    return response_content


In [2]:
def save_car_info(response_content):

    # Parse the JSON response
    try:
        json_match = re.search(r'\{.*?\}', response_content, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
            car_info = json.loads(json_str)
            brand = car_info.get("brand", "Unknown")
            color = car_info.get("color", "Unknown")
            condition = car_info.get("condition", "Unknown")

            print(f"Brand: {brand}")
            print(f"Color: {color}")
            print(f"Condition: {condition}")

            # Generate a new UUID for the id
            car_id = uuid.uuid4()

            # Connect to the Cassandra database (AstraDB)
            cloud_config = {
                'secure_connect_bundle': '/workspace/secure-connect-laetitiadb.zip'
            }
            auth_provider = PlainTextAuthProvider('token', 'AstraCS:pZksehWIKZuEKnnOOokneZOa:f28fd7765f4272dc4d4928c06ac1147ba0abd16a45ccb833458c37f3f978ac3b')
            cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
            session = cluster.connect('laetitia_astra')

            # Insert the data into the car_info table
            session.execute(
                "INSERT INTO car_info (id, brand, color, condition) VALUES (%s, %s, %s, %s)",
                (car_id, brand, color, condition)
            )

            print("Data saved to database successfully.")
        else:
            print("Failed to extract JSON from the response")

    except json.JSONDecodeError:
        print("Failed to parse the JSON response")


In [ ]:
image_path = "/workspace/00089.jpg"
response_content = analyse_car_image(image_path)
save_car_info(response_content)

In [ ]:
#writing test using copilot: how does the code handle bad input (no car)?


#use copilot to adjust analyse_car_image function to handle bad input (no car)

In [3]:
cloud_config = {
    'secure_connect_bundle': '/workspace/secure-connect-laetitiadb.zip'
}
auth_provider = PlainTextAuthProvider('token', 'AstraCS:pZksehWIKZuEKnnOOokneZOa:f28fd7765f4272dc4d4928c06ac1147ba0abd16a45ccb833458c37f3f978ac3b')
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect('laetitia_astra')
rows = session.execute("SELECT id, brand, color, condition FROM car_info")

# Print all rows
for row in rows:
    print(f"ID: {row.id}, Brand: {row.brand}, Color: {row.color}, Condition: {row.condition}")


ID: e4604d52-273f-4ac8-81fc-582300ea25e2, Brand: Volkswagen, Color: Red, Condition: good condition
ID: a9734835-0853-4137-83a9-9ef635152ad8, Brand: Volkswagen, Color: Red, Condition: good condition


In [6]:
cloud_config = {
    'secure_connect_bundle': '/workspace/secure-connect-laetitiadb.zip'
}
auth_provider = PlainTextAuthProvider('token', 'AstraCS:pZksehWIKZuEKnnOOokneZOa:f28fd7765f4272dc4d4928c06ac1147ba0abd16a45ccb833458c37f3f978ac3b')
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect('laetitia_astra')
session.execute("TRUNCATE car_info")